This notebook is for calculating distances, elevations and timings between Dovetrek checkpoints. It then produces a routecard for a set route.

###Contents:
* [Imports](#Imports)
* [Start SparkSession](#Start-SparkSession)
* [Set Parameters](#set-params)
* [Get Secrets](#get-secrets)
* [Fetch Checkpoint Information](#fetch-cp-info)
* [Fetch Distances from Filestore](#)
* [Calculate leg timings for each Checkpoint Combination](#cp-combo-times)
* [Pick a route](#pick-route)
* [Calculate route timings](#route-timings)
* [Calculate checkpoint arrival and departure times for route](#route-times)

### Imports

In [43]:
from pyspark.sql.functions import col, expr, first
from pyspark.sql import functions as F, DataFrame, Window, SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, FloatType, ArrayType, TimestampType
from datetime import datetime, time, timedelta
import requests
import time
import ipywidgets
import base64
import getpass
from IPython.display import clear_output
import builtins
import json
from io import BytesIO, StringIO
import itertools
from functools import reduce
import numpy as np

### Start SparkSession

In [44]:
try:
    SparkSession.active()
except:
    spark = SparkSession.builder \
        .appName("BinderPySpark") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .getOrCreate()
else:
    SparkSession.active()
finally:
    # Test if Spark is working
    df = spark.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "name"])
    df.show()
    df.unpersist()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



### Get available competition years

In [45]:
url = f"https://api.github.com/repos/liamj-f/Dovetrek/contents/CheckpointData"
params = {"ref": "main"}
response = requests.get(url, params=params)
response.json()
# Extract the 'name' field from each item in the JSON response
Year_List = [item['name'] for item in response.json() if 'name' in item]

# Extract the year using string slicing
Year_List = [filename.split('_')[1].split('.')[0] for filename in Year_List]

### Set Parameters

In [46]:
API_Service_List = ["Bing Maps","Google Maps", "OpenRouteService","Azure Maps & OpenTopoData"]

Competition_Year_Picker = ipywidgets.Dropdown(options=Year_List, value = '2025', description = 'Pick a competition year:')
API_Service_Picker = ipywidgets.Dropdown(options=API_Service_List, value = "Google Maps", description = 'Pick an API Service for distance & elevations:')
StopTimePicker = ipywidgets.IntSlider(value = 7
                                            , min = 0
                                            , max= 20
                                            , description = 'Pick a time in minutes to stop at checkpoints'
                                            , readout = True
                                          , step = 1)
NaismithPicker = ipywidgets.IntSlider(value = 10
                                            , min = 0
                                            , max= 20
                                            , description = 'Add minutes per 100m elevation gain'
                                            , readout = True
                                          , step = 1)
SpeedPicker = ipywidgets.FloatSlider(
                                          value=5.3,
                                          min=0,
                                          max=10.0,
                                          step=0.1,
                                          description='Hiking speed:',
                                          disabled=False,
                                          continuous_update=False,
                                          orientation='horizontal',
                                          readout=True,
                                          readout_format='.1f',
                                      )
StartTimePicker = ipywidgets.Text(
                                              description='Type a StartTime in the format HH:MM:SS',
                                              value = '10:00:00',
                                              disabled=False
                                          )
display(Competition_Year_Picker, API_Service_Picker, StopTimePicker, NaismithPicker, StartTimePicker, SpeedPicker)

# Create a button and output display area
run_button = ipywidgets.Button(description="Confirm Selection")
output = ipywidgets.Output()

def on_run_button_clicked(b):
    with output:
        clear_output()
        global Competition_Year_Picker, API_Service_Picker, StopTimePicker, NaismithPicker, StartTimePicker, SpeedPicker
        Competition_Year = Competition_Year_Picker.value
        API_Service = API_Service_Picker.value
        StopTimeAtCheckPoints = StopTimePicker.value
        AddMinutesPer100mHeight = NaismithPicker.value
        Speed = SpeedPicker.value
        StartTime = datetime.strptime(StartTimePicker.value, '%H:%M:%S')
        FinishTime = StartTime + timedelta(hours=7)

        print(f"Competition Year: {Competition_Year} \nAPI Service: {API_Service} \nStopTimeAtCheckPoints: {StopTimeAtCheckPoints} minutes \nAddMinutesPer100mHeight: {AddMinutesPer100mHeight} minutes \nSpeed: {Speed} km/h \nStartTime: {StartTime}")

run_button.on_click(on_run_button_clicked)
display(run_button, output)
###################

print("Make sure all of the parameters are set correctly before entering your GitHub Secrets repository Personal Access Token below")
GitHubPAT = getpass.getpass("Token:")

Dropdown(description='Pick a competition year:', index=4, options=('2017', '2018', '2019', '2024', '2025'), va…

Dropdown(description='Pick an API Service for distance & elevations:', index=1, options=('Bing Maps', 'Google …

IntSlider(value=7, description='Pick a time in minutes to stop at checkpoints', max=20)

IntSlider(value=10, description='Add minutes per 100m elevation gain', max=20)

Text(value='10:00:00', description='Type a StartTime in the format HH:MM:SS')

FloatSlider(value=5.3, continuous_update=False, description='Hiking speed:', max=10.0, readout_format='.1f')

Make sure all of the parameters are set correctly before entering your GitHub Secrets repository Personal Access Token below
Token:··········


### Check parameters correctly set

In [47]:
time.sleep(3)
print(f"Competition Year: {Competition_Year} \nAPI Service: {API_Service} \nStopTimeAtCheckPoints: {StopTimeAtCheckPoints} minutes \nAddMinutesPer100mHeight: {AddMinutesPer100mHeight} minutes \nSpeed: {Speed} km/h \nStartTime: {StartTime}")

Competition Year: 2025 
API Service: Google Maps 
StopTimeAtCheckPoints: 7 minutes 
AddMinutesPer100mHeight: 10 minutes 
Speed: 5.3 km/h 
StartTime: 1900-01-01 10:00:00


### Fetch Checkpoint information

In [48]:
# GitHub API URL for file content
url = f"https://api.github.com/repos/liamj-f/Dovetrek/contents/CheckpointData/Openings_{Competition_Year}.csv"

# Fetch file content
response = requests.get(url)
content = response.json()["content"]
decoded_content = base64.b64decode(content).decode("utf-8")
csv_lines = decoded_content.split("\n")
# Create an RDD from the list
rdd = spark.sparkContext.parallelize(csv_lines)
# Convert RDD to DataFrame
openings_df = spark.read.csv(rdd, header=True, inferSchema=True)

### List Filestore contents

In [49]:
url = f"https://api.github.com/repos/liamj-f/Dovetrek/contents/DataFrames"

params = {"ref": "FileStore"}

response = requests.get(url, params=params)

response.json()

[{'name': 'Distances_DF_2025_Azure Maps & OpenTopoData_2025-02-23.csv',
  'path': 'DataFrames/Distances_DF_2025_Azure Maps & OpenTopoData_2025-02-23.csv',
  'sha': '7d1d3a863ac353970da56c8576bc6fcb64642c78',
  'size': 8170,
  'url': 'https://api.github.com/repos/liamj-f/Dovetrek/contents/DataFrames/Distances_DF_2025_Azure%20Maps%20&%20OpenTopoData_2025-02-23.csv?ref=FileStore',
  'html_url': 'https://github.com/liamj-f/Dovetrek/blob/FileStore/DataFrames/Distances_DF_2025_Azure%20Maps%20&%20OpenTopoData_2025-02-23.csv',
  'git_url': 'https://api.github.com/repos/liamj-f/Dovetrek/git/blobs/7d1d3a863ac353970da56c8576bc6fcb64642c78',
  'download_url': 'https://raw.githubusercontent.com/liamj-f/Dovetrek/FileStore/DataFrames/Distances_DF_2025_Azure%20Maps%20%26%20OpenTopoData_2025-02-23.csv',
  'type': 'file',
  '_links': {'self': 'https://api.github.com/repos/liamj-f/Dovetrek/contents/DataFrames/Distances_DF_2025_Azure%20Maps%20&%20OpenTopoData_2025-02-23.csv?ref=FileStore',
   'git': 'http

### Get the Latest File

In [50]:
import fnmatch
import urllib
# Define the pattern with wildcard
pattern = "Distances_DF_2025_"+API_Service+"_*.csv"

# Filter files matching the pattern
matched_files = [file for file in response.json() if fnmatch.fnmatch(file["name"], pattern)]

# Sort by date in filename (YYYY-MM-DD at the end)
matched_files.sort(key=lambda x: x["name"].split("_")[-1], reverse=True)

# Get the latest matching file
latest_file = matched_files[0] if matched_files else None

if latest_file:
    print("Latest matching file:", latest_file["name"])
    print("Download URL:", latest_file["download_url"])
else:
    print("No matching files found.")

latest_file = latest_file['name']
latest_file = urllib.parse.quote(latest_file)


Latest matching file: Distances_DF_2025_Google Maps_2025-03-02.csv
Download URL: https://raw.githubusercontent.com/liamj-f/Dovetrek/FileStore/DataFrames/Distances_DF_2025_Google%20Maps_2025-03-02.csv


### Fetch Distances_DF csv to GitHub filestore

In [51]:

url = f"https://api.github.com/repos/liamj-f/Dovetrek/contents/DataFrames/{latest_file}"

params = {"ref": "FileStore"}
# Get the latest SHA (if the file exists)
response = requests.get(url, params=params)
content = response.json()["content"]
decoded_content = base64.b64decode(content).decode("utf-8")
csv_lines = decoded_content.split("\n")
# Create an RDD from the list
rdd = spark.sparkContext.parallelize(csv_lines)
# Convert RDD to DataFrame
Distances_DF = spark.read.csv(rdd, header=True, inferSchema=True)

display(Distances_DF)

DataFrame[StartCP: string, FinishCP: string, Distance: double, Height_Gain: double]

### Calculate leg timings for each Checkpoint Combination

In [52]:
Distances_DF = Distances_DF.withColumn("TimeInMinutes",(col("Distance")/ Speed)*60 + (col("Height_Gain") / AddMinutesPer100mHeight ) )
display(Distances_DF)

DataFrame[StartCP: string, FinishCP: string, Distance: double, Height_Gain: double, TimeInMinutes: double]

### Pick a Route

In [53]:
# Define the route and start time
route = ["Start","CP3", "E7U", "CP4", "E8", "E6", "CP5", "E5", "E4R", "CP6", "E9U","CP2","E2R","E10U", "E1", "CP1", "CP7", "E3", "Finish"]

### Calculate route timings

In [54]:
# Create a DataFrame for the route sequence
route_df = spark.createDataFrame([(route[i], route[i+1], i) for i in range(len(route)-1)],
                                 ["StartCP", "FinishCP", "RouteOrder"])

# Join with Distances_Df to get TimeInMinutes for each leg
timing_results_df = (route_df
      .join(Distances_DF, ["StartCP", "FinishCP"], "left")
      .orderBy("RouteOrder"))

# Use a window function to calculate cumulative time
window_spec = Window.orderBy("RouteOrder").rowsBetween(Window.unboundedPreceding, 0)
timing_results_df = timing_results_df.withColumn("CumulativeTime", F.sum("TimeInMinutes").over(window_spec))

# Show results
display(timing_results_df)

DataFrame[StartCP: string, FinishCP: string, RouteOrder: bigint, Distance: double, Height_Gain: double, TimeInMinutes: double, CumulativeTime: double]

### Calculate checkpoint arrival and departure times for route

In [55]:
# Define window specification to order rows by RouteOrder
window_spec = Window.orderBy("RouteOrder")

# Calculate the CumulativeTimeWithDwell (CumulativeTime + StopTimeAtCheckpoints)
timing_results_df = timing_results_df.withColumn(
    "CumulativeTimeWithDwell",
    F.col("CumulativeTime") + (F.col("RouteOrder") * F.lit(StopTimeAtCheckPoints))
)

# Calculate the LeavingTime (StartTime + CumulativeTimeWithDwell)
timing_results_df = timing_results_df.withColumn(
    "LeavingTime",
    F.to_timestamp(F.lit(StartTime)) + F.col("CumulativeTimeWithDwell").cast("int").cast("interval minute")
)

# Calculate the ArrivalTime (LeavingTime - StopTimeAtCheckpoints)
timing_results_df = timing_results_df.withColumn(
    "ArrivalTime",
    F.col("LeavingTime") - F.expr(f"INTERVAL {StopTimeAtCheckPoints} MINUTE")
)

# Show the results
display(timing_results_df)


DataFrame[StartCP: string, FinishCP: string, RouteOrder: bigint, Distance: double, Height_Gain: double, TimeInMinutes: double, CumulativeTime: double, CumulativeTimeWithDwell: double, LeavingTime: timestamp, ArrivalTime: timestamp]

In [58]:
timing_results_df.show()

+-------+--------+----------+--------+-----------+------------------+------------------+-----------------------+-------------------+-------------------+
|StartCP|FinishCP|RouteOrder|Distance|Height_Gain|     TimeInMinutes|    CumulativeTime|CumulativeTimeWithDwell|        LeavingTime|        ArrivalTime|
+-------+--------+----------+--------+-----------+------------------+------------------+-----------------------+-------------------+-------------------+
|  Start|     CP3|         0|     1.7|  63.561554| 25.60143841886792| 25.60143841886792|      25.60143841886792|1900-01-01 10:25:00|1900-01-01 10:18:00|
|    CP3|     E7U|         1|    4.51|  93.595764| 60.41618017358491| 86.01761859245283|      93.01761859245283|1900-01-01 11:33:00|1900-01-01 11:26:00|
|    E7U|     CP4|         2|   1.605|  18.484467| 20.01825802075472|106.03587661320755|     120.03587661320755|1900-01-01 12:00:00|1900-01-01 11:53:00|
|    CP4|      E8|         3|   2.014|   5.176193|23.317619299999997|129.353495913